In [1]:
%cd ..

/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3


/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import os

import pandas as pd
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from tqdm import tqdm

from augments import WikipediaCrawler
from prompts import load_template
from utils import DataVersionManager

/Users/chan/boostcamp-project/level2-nlp-generationfornlp-nlp-06-lv3/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
load_dotenv()
dvm = DataVersionManager()
wc = WikipediaCrawler()

PARAGRAPH = "paragraph"
QUESTION = "question"
CHOICES = "choices"
QUESTION_PLUS = "question_plus"
ANSWER = "answer"
REASONING = "reasoning"

In [ ]:
df = dvm.search_latest_experiments_data(1, False)
df.head()

,id,paragraph,question,choices,answer,question_plus,category,reasoning
0,generation-for-nlp-1825,우크라이나 사태를 둘러싸고 러시아에 대한 서방 국가들의 경제 제재가 가속화하면서 러...,우크라이나 사태와 관련하여 러시아에 대한 경제 제재를 가하고 있는 국가가 아닌 것은?,"['미국', '일본', '영국', '캐나다', '중국']",5,NaN,외적 추론,문제 분석 \n주어진 질문은 우크라이나 사태와 관련하여 러시아에 대한 경제 제재를...
1,generation-for-nlp-1055,그러는 동안 인디언들은 그들 주위를 조용히 찾아왔고 냉담한 모습을 보인 적도 있지만...,위 구절에서 분명히 알 수 있듯 1620년대에 매사추세츠가 된 곳의 유럽인은 순례자...,"['어업', '농업', '광산', '조선']",1,NaN,외적 추론,문제 분석 \n주어진 구절은 1620년대 매사추세츠 지역의 유럽인과 인디언 간의 ...
2,generation-for-nlp-521,얼레빗질 기계(Scribbling-Machines)로 인해 수천 명의 청원자가 일자...,이 문서는 무엇의 증거로 사용될 수 있습니까?,"['18세기 영국 노동자 계급 사이에 퍼진 혁명적 사상', '18세기 영국에서 시작...",3,NaN,외적 추론,문서 분석\n주어진 문서는 얼레빗질 기계의 도입으로 인해 수천 명의 노동자가 일자리...
3,generation-for-nlp-1053,"""한편으로는 노예 소유자가 모든 반대자보다 결정적인 이점을 갖고 있다. 이러한 이점...",프레데릭 더글라스의 발췌문이 가장 명백하게 보여주는 19세기 중반의 발전은 다음 중...,"['계약 하인을 아프리카 노예로 점진적으로 대체', '문화적 적응을 통한 아프리카 ...",3,NaN,외적 추론,문제 분석 \n프레데릭 더글라스의 발췌문은 19세기 중반의 노예제와 관련된 사회적...
4,generation-for-nlp-604,"""하지만 친애하는 팡글로스."" 캉디드가 말했다. ""내가 어찌 당신을 다시 볼 수 있...",볼테르가 캉디드를 통해 제시한 비평을 가장 긴밀히 공유하는 철학자는 누구입니까?,"['체사레 바카리아', '장 자크 루소', '애덤 스미스', '데이비드 흄']",4,NaN,외적 추론,"문제 분석 \n볼테르의 작품 ""캉디드""는 철학적 비판과 풍자를 통해 당시의 철학적..."


In [ ]:
prompt = PromptTemplate.from_template(
    template=load_template(file_name="wikipedia_search_keyword_with_reasoning.txt", template_type="keyword_extraction")
)

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=os.getenv("OPENAI_API_KEY"),
)

keyword_extractor = prompt | llm

In [ ]:
def extract_keyword(data: pd.Series) -> list[str]:
    """
    문제 해결에 필요한 키워드 5개를 추출하고 결과를 배열로 반환하는 함수

    Args:
        data (pd.Series): 제공된 데이터셋의 한 행(row), 문제 하나

    Returns:
        list[str]: 추출된 5개 키워드
    """
    response = keyword_extractor.invoke(
        {
            "paragraph": data[PARAGRAPH],
            "question": data[QUESTION],
            "question_plus": data[QUESTION_PLUS],
            "choices": data[CHOICES],
        },
    )
    response_split = response.content.split(",")
    keywords = [keyword.strip() for keyword in response_split]
    return keywords


tqdm.pandas()
sampled_df = df.sample(20, random_state=1004)
sampled_df["keywords"] = sampled_df.progress_apply(extract_keyword, axis=1)
sampled_df.head()

100%|██████████| 20/20 [00:17<00:00,  1.16it/s]


,id,paragraph,question,choices,answer,question_plus,category,reasoning,keywords
9,generation-for-nlp-437,1929년에 통학열차를 이용하던 한 일본인 학생이 한국인 여학생을 희롱한 사건이 일...,밑줄 친 ‘이 운동’에 대한 설명으로 옳은 것은?,"['진주에서 조선형평사가 창설되는 결과로 이어졌다.', '조선민립대학 설립운동이 시...",3,NaN,외적 추론,문제 해결 과정\n\n배경 분석\n지문은 1929년에 발생한 일본인 학생의 한국인 ...,"[일제, 학생, 시위, 동맹휴학, 차별정책]"
12,generation-for-nlp-1065,그들은 우리에게 이 새로운 소유물의 통치 방법을 묻습니다. 저는 대답하겠습니다. 현...,비버리지의 연설 주제와 가장 직접적으로 비견되는 것은 다음 중 무엇인가?,"['먼로 독트린', '봉쇄', '명백한 운명', '국가의 권리에 대한 이데올로기']",3,NaN,외적 추론,비버리지의 연설 주제 분석\n\n앨버트 비버리지의 연설은 미국의 제국주의적 확장을 ...,"[국기의 행진, 미국의 통치, 명백한 운명, 자치 경험, 보호령]"
16,generation-for-nlp-1001,"""기혼 여성의 재산을 보다 효과적으로 보호하기 위한 법률:\n""§1. 앞으로 결혼할...",이 지문에 나타난 정보는 무엇입니까?,"['투표권에 대한 접근권을 둘러싼 토론', '결혼법에서 연방정부의 역할을 둘러싼 토...",4,NaN,외적 추론,이 지문은 기혼 여성의 재산 보호에 관한 법률을 다루고 있습니다. 법률의 내용은 기...,"[기혼 여성, 재산 보호, 부동산, 법률, 남편의 채무]"
18,generation-for-nlp-1188,"오, 수치스럽도다, 불쌍한 겨울의 왕이여! 그대는 도대체 무슨 짓을 벌인 것인가? ...",이 노래에 깔려 있는 중요한 이슈는 무엇입니까?,"['왕위계승의 규칙', '종교적 갈등', '군사적 교착상태', '사회적 계급 갈등']",2,NaN,외적 추론,"이 노래의 배경과 내용을 분석해보면, 주된 이슈는 **종교적 갈등**입니다. \n\...","[왕위계승, 페르디난트, 겨울의 왕, 수치, 경멸]"
14,generation-for-nlp-1139,종교적 직업과 예배에서 양심의 자유가 제한된다면 시민의 자유를 최대한 누리더라도 진...,윌리엄 펜은 박해를 받는 다음 종교 단체 중 어느 곳의 일원이었습니까?,"['로마 가톨릭', '청교도', '퀘이커', '영국 성공회']",3,NaN,외적 추론,"문제 분석 \n윌리엄 펜은 종교적 자유와 양심의 자유를 강조한 인물로, 그의 사상...","[양심의 자유, 전능하신 하나님, 종교적 예배, 퀘이커, 윌리엄 펜]"


In [ ]:
sampled_df.to_csv(
    dvm.update_file_path(1, 1, "minor", is_experiment=False, save_in_experiment=True), index=False
)  # 1차 중간 결과

추출된 키워드가 위키피디아에 존재하는지 여부 확인

- 키워드 추출 결과가 유효한지 확인하기 위한 작업입니다.


In [ ]:
# 키워드를 유용하게 다루기 위해 키워드 별로 칼럼으로 분리
keywords_expanded = pd.DataFrame(sampled_df["keywords"].tolist(), index=sampled_df.index)
keywords_expanded.columns = [f"keyword_{i+1}" for i in range(keywords_expanded.shape[1])]
new_df = pd.concat([sampled_df.drop(columns=["keywords"]), keywords_expanded], axis=1)

In [ ]:
# 위키피디아 페이지가 있는지 키워드 별로 확인
for col in keywords_expanded.columns:
    new_df[f"{col}_exists"] = pd.Series()
    new_df[f"{col}_page"] = pd.Series()

for idx, row in tqdm(new_df.iterrows()):
    for col in keywords_expanded.columns:
        result = wc.is_exists(row[col])
        new_df.loc[idx, f"{col}_exists"] = result[0]
        new_df.loc[idx, f"{col}_page"] = result[1] if result[0] else None

new_df.head()

20it [01:06,  3.31s/it]


,id,paragraph,question,choices,answer,question_plus,category,reasoning,keyword_1,keyword_2,...,keyword_1_exists,keyword_1_page,keyword_2_exists,keyword_2_page,keyword_3_exists,keyword_3_page,keyword_4_exists,keyword_4_page,keyword_5_exists,keyword_5_page
9,generation-for-nlp-437,1929년에 통학열차를 이용하던 한 일본인 학생이 한국인 여학생을 희롱한 사건이 일...,밑줄 친 ‘이 운동’에 대한 설명으로 옳은 것은?,"['진주에서 조선형평사가 창설되는 결과로 이어졌다.', '조선민립대학 설립운동이 시...",3,NaN,외적 추론,문제 해결 과정\n\n배경 분석\n지문은 1929년에 발생한 일본인 학생의 한국인 ...,일제,학생,...,True,"대일본제국(일본어: 大日本帝国 다이닛폰테이코쿠[*], 일본어 구자체: 大日本帝󠄁國)...",True,"학생(學生, 영어: student)은 학교에 다니면서 교육을 받는 사람을 뜻하는 말...",True,"정치 집회, 즉 넓은 의미에서의 집회(集會)는 다수의 사람들이 일정한 공동의 목적을...",True,동맹휴학(同盟休學)은 학생들이 학교 수업을 듣는 것을 집단적으로 거부하는 항의행동이...,False,None
12,generation-for-nlp-1065,그들은 우리에게 이 새로운 소유물의 통치 방법을 묻습니다. 저는 대답하겠습니다. 현...,비버리지의 연설 주제와 가장 직접적으로 비견되는 것은 다음 중 무엇인가?,"['먼로 독트린', '봉쇄', '명백한 운명', '국가의 권리에 대한 이데올로기']",3,NaN,외적 추론,비버리지의 연설 주제 분석\n\n앨버트 비버리지의 연설은 미국의 제국주의적 확장을 ...,국기의 행진,미국의 통치,...,False,None,False,None,True,"명백한 운명(영어: Manifest Destiny)이란, 제임스 매디슨이 미국 대통...",False,None,True,두 개의 국가가 보호조약을 체결하여 한편의 나라가 다른편의 나라를 외부의 공격으로부...
16,generation-for-nlp-1001,"""기혼 여성의 재산을 보다 효과적으로 보호하기 위한 법률:\n""§1. 앞으로 결혼할...",이 지문에 나타난 정보는 무엇입니까?,"['투표권에 대한 접근권을 둘러싼 토론', '결혼법에서 연방정부의 역할을 둘러싼 토...",4,NaN,외적 추론,이 지문은 기혼 여성의 재산 보호에 관한 법률을 다루고 있습니다. 법률의 내용은 기...,기혼 여성,재산 보호,...,False,None,False,None,True,"부동산(不動産, 영어: real estate, immovable property)은...",True,"법률(法律, 독일어: Gesetz)은 주로 입법부의 심의와 의결을 거치고, 국가 원...",False,None
18,generation-for-nlp-1188,"오, 수치스럽도다, 불쌍한 겨울의 왕이여! 그대는 도대체 무슨 짓을 벌인 것인가? ...",이 노래에 깔려 있는 중요한 이슈는 무엇입니까?,"['왕위계승의 규칙', '종교적 갈등', '군사적 교착상태', '사회적 계급 갈등']",2,NaN,외적 추론,"이 노래의 배경과 내용을 분석해보면, 주된 이슈는 **종교적 갈등**입니다. \n\...",왕위계승,페르디난트,...,True,왕위 계승(王位継承)은 일반적으로 왕위(군주의 지위)를 왕태자가 이어받은 것을 말한...,True,페르디난트는 다음 등을 가리킨다.\n신성 로마 제국\n\n페르디난트 1세(Ferdi...,False,None,True,수치는 다음의 뜻이 있다.\n\n수치는 자기의 약점이나 잘못 또는 무가치(無價値)함...,True,"경멸(輕蔑)은 남을 깔보는 감정이다. 폴 에크만의 6가지 기본 감정인 노여움, 혐오..."
14,generation-for-nlp-1139,종교적 직업과 예배에서 양심의 자유가 제한된다면 시민의 자유를 최대한 누리더라도 진...,윌리엄 펜은 박해를 받는 다음 종교 단체 중 어느 곳의 일원이었습니까?,"['로마 가톨릭', '청교도', '퀘이커', '영국 성공회']",3,NaN,외적 추론,"문제 분석 \n윌리엄 펜은 종교적 자유와 양심의 자유를 강조한 인물로, 그의 사상...",양심의 자유,전능하신 하나님,...,True,"양심의 자유(良心- 自由, 영어: freedom of conscience)란 인간의...",False,None,False,None,True,"종교친우회(宗敎親友會, 영어: Religious Society of Friends)...",True,이름이 윌리엄 펜인 사람으로는 다음이 있다.\n\n윌리엄 펜 (1621년) : 영국...


In [19]:
# _exists 열만 추출
exists_columns = [col for col in new_df.columns if col.endswith("_exists")]

true_count = new_df[exists_columns].sum().sum()  # True 값의 총합
false_count = (new_df[exists_columns] == False).sum().sum()  # False 값의 총합

true_count, false_count

(70, 30)

In [ ]:
new_df.to_csv(dvm.update_file_path(1, 1, "minor", is_experiment=False, save_in_experiment=True), index=False)